In [6]:
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from cem.match import match
from cem.imbalance import L1, L2
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import r2_score

from utils import stata_dir, data_dir, code_dir, negative_outcomes, democracies, costs_raw

In [7]:
database = pd.read_csv(os.path.join(stata_dir, 'Full_Database.csv'))
database['l_pwd_a'] = np.log(database['PWD_A'])
database['l_pwd_g'] = np.log(database['PWD_G'])
database['l_pwd_m'] = np.log(database['PWD_M'])

In [9]:

outcomes = [
    'GoodsRatio',
    'PrivateorPublicGoods',
    'EliteWantCommonGood',
    'PublicWantCommonGood',
    'TransparentLaws',
    'JudicialIndep',
    'FreeExpression',
    'Corruption',
    'ExecutiveCorrupt',
    'PublicSectorTheft',
    'Clientelism',
    'CorruptionTI',
    'StateOwnsEconomy',
    'JudgeCorrupt',
    'RespectConstitution',
    'RuleofLaw',
    'FreeElections',
    'FreeMovement',
    'CivilLiberties',
    'PolRights',
    'ReligiousFreedom',
    'JudicialIndep',
    'PropertyRights',
    'MediaCensored',
    'PackCourts',
    'Torture',
    'SlaveLabor',
    'CampusFree',
    'AcademicFreedom',
    # 'EducExpend',
    'SecondarySchool',
    'InfantMortality',
    'HealthCare',
    'LifeExpect',
    'Vaccination',
    'CleanWater',
    'Hygiene',
    # 'HealthExpend',
    'FoodConsumption',
    'ElectricAccess',
    'Transparencyindex',
    'PressFreedom',
]

d_controls = {
    'W4': (pd.cut, {'bins': 6}),
    # 'support': (pd.qcut, {'q': 4}),
    # 'W_old': (pd.qcut, {'q': 4}),
    # 'normpolity': (pd.qcut, {'q': 4}),
    # 'Dem6': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'e_boix_regime': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'Przeworski': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'gwf_party': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'gwf_military': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'gwf_monarchy': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'gwf_personal': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
    # 'gwf_demo': (pd.cut, {'bins': 4}), # throws "Bin edges must be unique" otherwise
}

pop_densities = {
    'l_pwd_a': (pd.qcut, {'q': 4}),
    'l_pwd_g': (pd.qcut, {'q': 4}),
    'l_pwd_m': (pd.qcut, {'q': 4}),
    'wb_urbpop': (pd.cut, {'bins': 4}),
}

resemblances = {
    'Elr': (pd.qcut, {'q': 4}),
    'Epr': (pd.qcut, {'q': 4}),
    'Err': (pd.qcut, {'q': 4}),
    'Egr': (pd.cut, {'bins': 2}), # Most countries have Egr=1
}

educ = {
    'e_peaveduc': (pd.cut, {'bins': 6}),
}

controls = {
    'e_migdppcln': (pd.cut, {'bins': 6}),
    'logpop': (pd.cut, {'bins': 6}),
}

eras = [
    database['year'] <= 1890,
    (1890 < database['year']) & (database['year'] <= 1945),
    (1945 < database['year']) & (database['year'] <= 2000),
    2000 < database['year']
]
database['era'] = np.select(eras, [1,2,3,4])

all_controls = [
    'era',
    *controls.keys(),
    # 'l_pwd_a',
    # 'Epr',
    list(educ.keys())[0],
]

schema = {
    **pop_densities,
    **resemblances,
    **educ,
    **controls,
    **d_controls,
}

for c in {*all_controls, *schema.keys(), *outcomes}:
    database[c] = pd.to_numeric(database[c])

for t in [*democracies, *costs_raw]:
    if len(database[t].value_counts()) == 2:
        m = 0.5
    else:
        m = database[t].median()

    database[f'{t}_coarse'] = database[t].apply(lambda x: int(x >= m))
    # print(d, 'median:', m)

def coarsen(x: pd.Series):
    if x.name in schema:
        return schema[x.name][0](np.array(x), **schema[x.name][1])
    # if x.name == 'year':
    #     return x
    else:
        return x

coarse_db = database.apply(coarsen)

In [15]:
m_res = []

for out in outcomes[:]:
    # for t in [*democracies, *costs_raw]:
    for t in [ *costs_raw]:

        if t == 'e_peaveduc':
            continue

        treatment = f'{t}_coarse'

        c = [*all_controls]
        c.append(treatment)

        if t in costs_raw: # control for democracy in p
            c.append('W4')
        elif t in democracies:
            c.append('Epr')

        db = coarse_db[[*c, out]] # lean
        valuefull = ~db.isnull().any(axis=1)
        db = db[valuefull]
        y = db[out]
        db = db.drop(columns=[out])

        weights = match(db, treatment)
        n = (weights > 0).sum()

        if L1(db, treatment, weights) > 2e-10:
            raise Exception('Imbalanced panel!')
        if L2(db, treatment, weights) > 2e-10:
            raise Exception('Imbalanced panel!')

        model = sm.WLS(y, sm.add_constant(database[valuefull][c]), weights=weights)
        results = model.fit()
        r_bt = pd.DataFrame({
            'beta': results.params,
            'tvalue': results.tvalues
        })
        m_res.append([out, treatment, *r_bt.loc[treatment], n])

model_results = pd.DataFrame(m_res, columns=['outcome', 'model', 'beta', 'tstat', 'N'])

KeyboardInterrupt: 

In [14]:
from scipy.stats import norm


def t2p(t: float | pd.Series):
    return 1 - norm.cdf(abs(t))

def calc_bh(row: pd.DataFrame):
    if row['outcome'] in negative_outcomes:
        return int(row['beta'] < 0) # type: ignore
    else:
        return int(row['beta'] > 0) # type: ignore

model_results['bh'] = model_results.apply(calc_bh, axis=1)

model_results['pstat'] = t2p(model_results['tstat'])

model_results['p100'] = model_results['bh'] & (model_results['pstat'] < 0.100)
model_results['p050'] = model_results['bh'] & (model_results['pstat'] < 0.050)
model_results['p010'] = model_results['bh'] & (model_results['pstat'] < 0.010)
model_results['p001'] = model_results['bh'] & (model_results['pstat'] < 0.001)

model_results['absbeta'] = model_results['bh'] * model_results['beta'].abs()

per_model_r = model_results.groupby('model')[[
    'bh', 
    'p100', 'p050', 'p010', 'p001',
]].sum()
per_model_r['beta'] = model_results.groupby('model')['absbeta'].mean()
per_model_r = per_model_r.reset_index()

per_model_r.to_csv(os.path.join(code_dir, 'CEM_Analysis.csv'), index=False)
per_model_r

,model,bh,p100,p050,p010,p001,beta
0,Dem6_coarse,36,35,34,34,34,1.577863
1,Egr_coarse,25,20,18,15,13,0.173688
2,Elr_coarse,37,34,33,30,27,0.988507
3,Epr_coarse,38,36,36,36,36,0.755075
4,Err_coarse,39,38,38,38,35,0.760840
5,Przeworski_coarse,36,36,35,35,33,1.156420
6,W4_coarse,35,34,34,33,33,1.658822
7,W_old_coarse,33,33,32,31,31,0.853645
8,e_boix_regime_coarse,36,35,33,33,32,1.260271
9,gwf_demo_coarse,37,34,34,34,33,1.089192


In [13]:
from utils import all_outcomes
print(len(all_outcomes))

42
